# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key
gmaps.configure(api_key=g_key)

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
city_data = pd.read_csv("city_data.csv", dtype="object", encoding="utf-8")
city_data

,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
0,novoagansk,61.94,76.66,76.86,29,100,18.88,RU,2020-05-09
1,bubaque,11.28,-15.83,78.13,73,92,12.46,GW,2020-05-09
2,dois corregos,-22.37,-48.38,55.0,90,21,5.01,BR,2020-05-09
3,port alfred,-33.59,26.89,63.0,97,0,1.34,ZA,2020-05-09
4,rikitea,-23.12,-134.97,75.36,72,28,6.4,PF,2020-05-09
...,...,...,...,...,...,...,...,...,...
557,krasnokamensk,50.1,118.04,62.78,31,90,4.72,RU,2020-05-09
558,pueblo rico,5.24,-76.04,66.81,88,100,1.45,CO,2020-05-09
559,lompoc,34.64,-120.46,61.0,100,90,5.82,US,2020-05-09
560,mananara,-16.17,49.77,79.99,71,48,7.63,MG,2020-05-09


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [8]:
# Customize the size of the figure
locations = city_data[["Latitude", "Longitude"]].astype(float)
humidity = city_data["Humidity"].astype(float)

In [7]:
figure_layout = {
    'width': '800px',
    'height': '400px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(layout=figure_layout)
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius = 1)
fig.add_layer(heat_layer)

fig

Figure(layout=FigureLayout(border='1px solid black', height='400px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [19]:
city_data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 562 entries, 0 to 561
Data columns (total 9 columns):
 #   Column           Non-Null Count  Dtype 
---  ------           --------------  ----- 
 0   City             562 non-null    object
 1   Latitude         562 non-null    object
 2   Longitude        562 non-null    object
 3   Max Temperature  562 non-null    object
 4   Humidity         562 non-null    object
 5   Cloudiness       562 non-null    object
 6   Wind Speed       562 non-null    object
 7   Country          559 non-null    object
 8   Date             562 non-null    object
dtypes: object(9)
memory usage: 39.6+ KB


In [34]:
city_data[(city_data["Max Temperature"].astype(float)<80) & (city_data["Cloudiness"].astype(float)<10)]
# city_data[city_data["Cloudiness"].astype(float)<10]



,City,Latitude,Longitude,Max Temperature,Humidity,Cloudiness,Wind Speed,Country,Date
3,port alfred,-33.59,26.89,63.0,97,0,1.34,ZA,2020-05-09
7,irpa irpa,-17.72,-66.27,48.2,66,0,1.03,BO,2020-05-09
10,qaanaaq,77.48,-69.36,5.29,77,0,3.85,GL,2020-05-09
20,qaqortoq,60.72,-46.03,50.0,19,0,21.92,GL,2020-05-09
25,cherskiy,68.75,161.3,37.6,78,0,6.08,RU,2020-05-09
...,...,...,...,...,...,...,...,...,...
525,kaffrine,14.11,-15.55,77.0,69,6,4.7,SN,2020-05-09
540,bobrovka,54.05,61.75,78.21,29,0,16.4,RU,2020-05-09
543,umea,63.83,20.26,42.01,60,0,14.99,SE,2020-05-09
555,ostersund,63.18,14.64,35.6,64,0,6.93,SE,2020-05-09


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [ ]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [ ]:
# Add marker layer ontop of heat map


# Display figure
